# 時系列データの異常検知

このノートブックでは、時系列データの異常検知手法を学びます。

## 時系列異常検知とは？

**時系列異常検知**は、時間的な順序を持つデータから、正常なパターンから外れた異常を検出する手法です。

### なぜ時系列データに特別な手法が必要なのか？

1. **時間的な依存関係**: 過去の値が現在の値に影響する
2. **トレンド**: 長期的な増加・減少傾向
3. **季節性**: 周期的なパターン（日次、週次、月次など）
4. **突発的な変化**: スパイク、ドロップなど

### 静的データ vs 時系列データ

| 特徴 | 静的データ | 時系列データ |
|------|-----------|-------------|
| データ形式 | テーブル（行×列） | 時刻 + 値 |
| 順序 | 関係なし | **重要** |
| 依存関係 | 独立 | **時間的依存** |
| 異常検知 | Isolation Forest、LOF | 統計的手法、深層学習 |

### サイバー攻撃検知での応用

- **DDoS攻撃**: トラフィックの急激な増加
- **ポートスキャン**: 短時間での連続アクセス
- **データ漏洩**: 通常と異なるデータ転送量
- **異常ログイン**: 時間帯・頻度の異常

## 1. ライブラリのインポート

In [ ]:
# データ処理
import numpy as np
import pandas as pd

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns

# 機械学習（異常検知）
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

# 統計
from scipy import stats

# 設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.random.seed(42)

%matplotlib inline

print("ライブラリのインポート完了！")

## 2. 時系列異常検知の手法

### 主要な手法

#### 1. **統計的手法**
- **移動平均 (Moving Average)**: 過去N期間の平均からの乖離
- **標準偏差 (Standard Deviation)**: 平均 ± 3σ から外れたら異常
- **Z-Score**: 標準化された値が閾値を超えたら異常

#### 2. **ウィンドウベース手法**
- 時系列をウィンドウ（スライディングウィンドウ）に分割
- 各ウィンドウの特徴量を抽出
- Isolation Forestなどで異常検知

#### 3. **深層学習手法**（参考）
- **LSTM Autoencoder**: 正常パターンを学習し、再構成誤差で異常判定
- **GRU Autoencoder**: LSTMの軽量版
- **Transformer**: 最新のアーキテクチャ

このノートブックでは、**1と2の手法**を実装します。

## 3. データの生成

まず、シンプルな正弦波データで異常検知を実装します。

In [ ]:
# 正弦波データの生成
def generate_sine_wave(n_samples=1000, noise_level=0.1):
    """
    正弦波データを生成
    
    Parameters:
    - n_samples: サンプル数
    - noise_level: ノイズの強度
    """
    x = np.linspace(0, 50, n_samples)
    y = np.sin(x) + np.random.normal(0, noise_level, n_samples)
    return x, y

# 異常データの生成（スパイクを追加）
def add_anomalies(data, n_anomalies=10, spike_magnitude=3):
    """
    データに異常（スパイク）を追加
    
    Parameters:
    - data: 元のデータ
    - n_anomalies: 異常の数
    - spike_magnitude: スパイクの大きさ
    """
    data_with_anomalies = data.copy()
    anomaly_indices = np.random.choice(len(data), n_anomalies, replace=False)
    
    for idx in anomaly_indices:
        data_with_anomalies[idx] += spike_magnitude * np.random.choice([-1, 1])
    
    return data_with_anomalies, anomaly_indices

# データ生成
time, normal_data = generate_sine_wave(n_samples=1000, noise_level=0.1)
test_data, anomaly_indices = add_anomalies(normal_data, n_anomalies=15, spike_magnitude=3)

print(f"正常データのサイズ: {normal_data.shape}")
print(f"テストデータのサイズ: {test_data.shape}")
print(f"異常の数: {len(anomaly_indices)}")
print(f"異常の位置（最初の10個）: {sorted(anomaly_indices)[:10]}")

In [ ]:
# データの可視化
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 正常データ
axes[0].plot(time, normal_data, label='Normal Data', color='blue', alpha=0.7)
axes[0].set_title('Normal Sine Wave', fontsize=14)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 異常データ
axes[1].plot(time, test_data, label='Test Data', color='blue', alpha=0.7)
axes[1].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=100, label='True Anomalies', zorder=5)
axes[1].set_title('Sine Wave with Anomalies (Spikes)', fontsize=14)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 手法1: 統計的異常検知

### 4.1 移動平均と標準偏差による検知

**仕組み:**
1. 移動平均（MA）を計算: 過去N期間の平均
2. 移動標準偏差（MSD）を計算: 過去N期間の標準偏差
3. 閾値を設定: `MA ± k × MSD` (通常 k=3)
4. 閾値を超えたら異常と判定

**メリット:**
- シンプルで理解しやすい
- 計算が高速
- リアルタイム処理に適している

**デメリット:**
- 複雑なパターンは検出困難
- ウィンドウサイズの選択が重要

In [ ]:
def detect_anomalies_statistical(data, window_size=20, k=3):
    """
    移動平均と標準偏差を使った異常検知
    
    Parameters:
    - data: 時系列データ
    - window_size: 移動平均のウィンドウサイズ
    - k: 標準偏差の倍数（通常3）
    
    Returns:
    - anomalies: 異常フラグ (True/False)
    - ma: 移動平均
    - upper_bound: 上限閾値
    - lower_bound: 下限閾値
    """
    # DataFrameに変換
    df = pd.DataFrame({'value': data})
    
    # 移動平均と移動標準偏差
    df['ma'] = df['value'].rolling(window=window_size, center=False).mean()
    df['mstd'] = df['value'].rolling(window=window_size, center=False).std()
    
    # 閾値の計算
    df['upper_bound'] = df['ma'] + k * df['mstd']
    df['lower_bound'] = df['ma'] - k * df['mstd']
    
    # 異常判定
    df['anomaly'] = (df['value'] > df['upper_bound']) | (df['value'] < df['lower_bound'])
    
    return df['anomaly'].values, df['ma'].values, df['upper_bound'].values, df['lower_bound'].values

# 異常検知の実行
WINDOW_SIZE = 30
K_SIGMA = 3

stat_anomalies, ma, upper_bound, lower_bound = detect_anomalies_statistical(
    test_data, 
    window_size=WINDOW_SIZE, 
    k=K_SIGMA
)

print(f"検出された異常数: {stat_anomalies.sum()}")
print(f"異常の割合: {stat_anomalies.sum() / len(stat_anomalies) * 100:.2f}%")

In [ ]:
# 結果の可視化
plt.figure(figsize=(14, 7))

# 元のデータ
plt.plot(time, test_data, label='Data', color='blue', alpha=0.7, linewidth=1.5)

# 移動平均
plt.plot(time, ma, label='Moving Average', color='green', linewidth=2)

# 閾値
plt.plot(time, upper_bound, label='Upper Bound (MA + 3σ)', color='orange', 
         linestyle='--', linewidth=1.5)
plt.plot(time, lower_bound, label='Lower Bound (MA - 3σ)', color='orange', 
         linestyle='--', linewidth=1.5)

# 真の異常
plt.scatter(time[anomaly_indices], test_data[anomaly_indices], 
            color='red', s=150, label='True Anomalies', zorder=5, marker='o', 
            edgecolors='black', linewidths=2)

# 検出された異常
detected_indices = np.where(stat_anomalies)[0]
plt.scatter(time[detected_indices], test_data[detected_indices], 
            color='yellow', s=80, label='Detected Anomalies', zorder=4, 
            marker='x', linewidths=2)

plt.title('Statistical Anomaly Detection (Moving Average + 3σ)', fontsize=14)
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 評価
# 真のラベルを作成（異常=True, 正常=False）
true_labels = np.zeros(len(test_data), dtype=bool)
true_labels[anomaly_indices] = True

# NaNを正常として扱う（移動平均の初期値）
stat_anomalies_clean = np.nan_to_num(stat_anomalies, nan=False)

# 評価指標
precision_stat = precision_score(true_labels, stat_anomalies_clean, zero_division=0)
recall_stat = recall_score(true_labels, stat_anomalies_clean, zero_division=0)
f1_stat = f1_score(true_labels, stat_anomalies_clean, zero_division=0)

print("=== 統計的手法の評価 ===")
print(f"Precision: {precision_stat:.4f}")
print(f"Recall: {recall_stat:.4f}")
print(f"F1-Score: {f1_stat:.4f}")

print("\nClassification Report:")
print(classification_report(true_labels, stat_anomalies_clean, 
                            target_names=['Normal', 'Anomaly'], zero_division=0))

### 4.2 Z-Scoreによる異常検知

**Z-Score**は、データが平均からどれだけ離れているかを標準偏差の単位で表します。

```
Z = (x - μ) / σ
```

- **|Z| > 3**: 異常と判定（99.7%信頼区間外）
- **|Z| > 2**: やや異常（95%信頼区間外）

In [ ]:
# Z-Scoreの計算
z_scores = np.abs(stats.zscore(test_data))

# 閾値を設定（|Z| > 3）
threshold_z = 3
zscore_anomalies = z_scores > threshold_z

print(f"検出された異常数: {zscore_anomalies.sum()}")
print(f"異常の割合: {zscore_anomalies.sum() / len(zscore_anomalies) * 100:.2f}%")

# 評価
precision_z = precision_score(true_labels, zscore_anomalies, zero_division=0)
recall_z = recall_score(true_labels, zscore_anomalies, zero_division=0)
f1_z = f1_score(true_labels, zscore_anomalies, zero_division=0)

print("\n=== Z-Score手法の評価 ===")
print(f"Precision: {precision_z:.4f}")
print(f"Recall: {recall_z:.4f}")
print(f"F1-Score: {f1_z:.4f}")

In [ ]:
# Z-Scoreの可視化
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 元のデータ
axes[0].plot(time, test_data, label='Data', color='blue', alpha=0.7)
axes[0].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5)
detected_z = np.where(zscore_anomalies)[0]
axes[0].scatter(time[detected_z], test_data[detected_z], 
                color='yellow', s=80, label='Detected (Z-Score)', zorder=4, marker='x')
axes[0].set_title('Z-Score Anomaly Detection', fontsize=14)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Z-Score
axes[1].plot(time, z_scores, label='Z-Score', color='purple', alpha=0.7)
axes[1].axhline(threshold_z, color='red', linestyle='--', label=f'Threshold (|Z| = {threshold_z})')
axes[1].scatter(time[detected_z], z_scores[detected_z], 
                color='red', s=100, label='Detected Anomalies', zorder=5)
axes[1].set_title('Z-Score over Time', fontsize=14)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('|Z-Score|')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 手法2: ウィンドウベース + Isolation Forest

時系列データをウィンドウ（スライディングウィンドウ）に分割し、各ウィンドウから特徴量を抽出してIsolation Forestで異常検知します。

### ウィンドウとは？

連続したデータポイントの塊です。

```
元データ: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
window_size=3

ウィンドウ1: [1, 2, 3]
ウィンドウ2: [2, 3, 4]
ウィンドウ3: [3, 4, 5]
...
```

### 特徴量の抽出

各ウィンドウから統計的特徴量を計算:
- 平均 (mean)
- 標準偏差 (std)
- 最小値 (min)
- 最大値 (max)
- 範囲 (range = max - min)
- 勾配 (slope): 線形回帰の傾き

In [ ]:
def create_windowed_features(data, window_size=30):
    """
    時系列データからウィンドウベースの特徴量を抽出
    
    Parameters:
    - data: 1次元時系列データ
    - window_size: ウィンドウサイズ
    
    Returns:
    - features: (サンプル数, 特徴量数)の2次元配列
    """
    features = []
    
    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        
        # 統計的特徴量
        mean = np.mean(window)
        std = np.std(window)
        min_val = np.min(window)
        max_val = np.max(window)
        range_val = max_val - min_val
        
        # 勾配（線形回帰の傾き）
        x = np.arange(window_size)
        slope = np.polyfit(x, window, 1)[0]
        
        features.append([mean, std, min_val, max_val, range_val, slope])
    
    return np.array(features)

# ウィンドウサイズの設定
WINDOW_SIZE = 30

# 正常データから特徴量を抽出（訓練用）
X_train_features = create_windowed_features(normal_data, window_size=WINDOW_SIZE)

# テストデータから特徴量を抽出
X_test_features = create_windowed_features(test_data, window_size=WINDOW_SIZE)

print(f"訓練データの形状: {X_train_features.shape}")
print(f"テストデータの形状: {X_test_features.shape}")
print(f"\n特徴量: [mean, std, min, max, range, slope]")
print(f"\n最初のウィンドウの特徴量:")
print(X_test_features[0])

In [ ]:
# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.transform(X_test_features)

print(f"標準化後の訓練データの形状: {X_train_scaled.shape}")
print(f"標準化後のテストデータの形状: {X_test_scaled.shape}")

In [ ]:
# Isolation Forestによる異常検知
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 異常の割合を5%と仮定
    random_state=42
)

# 訓練データで学習
iso_forest.fit(X_train_scaled)

# テストデータで予測（-1: 異常, 1: 正常）
iso_predictions = iso_forest.predict(X_test_scaled)
iso_anomalies_window = iso_predictions == -1

# 異常スコア
iso_scores = iso_forest.score_samples(X_test_scaled)

print(f"検出された異常ウィンドウ数: {iso_anomalies_window.sum()}")
print(f"異常の割合: {iso_anomalies_window.sum() / len(iso_anomalies_window) * 100:.2f}%")

In [ ]:
# ウィンドウレベルの異常を元の時系列に変換
# ウィンドウが異常なら、そのウィンドウに含まれる全ての時点を異常としてマーク
iso_anomalies_timeseries = np.zeros(len(test_data), dtype=bool)

for i, is_anomaly in enumerate(iso_anomalies_window):
    if is_anomaly:
        # ウィンドウ全体を異常としてマーク
        start = i
        end = i + WINDOW_SIZE
        iso_anomalies_timeseries[start:end] = True

print(f"異常とマークされた時点の数: {iso_anomalies_timeseries.sum()}")

In [ ]:
# 結果の可視化
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 元のデータと異常検知結果
axes[0].plot(time, test_data, label='Data', color='blue', alpha=0.7, linewidth=1.5)
axes[0].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5, 
                edgecolors='black', linewidths=2)

# 検出された異常領域を塗りつぶし
for i, is_anomaly in enumerate(iso_anomalies_window):
    if is_anomaly:
        start = i
        end = i + WINDOW_SIZE
        axes[0].axvspan(time[start], time[min(end, len(time)-1)], 
                       alpha=0.3, color='yellow')

axes[0].set_title('Window-based Isolation Forest Anomaly Detection', fontsize=14)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 異常スコア
# ウィンドウの中心時刻を計算
window_centers = time[:len(iso_scores)] + WINDOW_SIZE // 2
axes[1].plot(window_centers, iso_scores, label='Anomaly Score', color='green', alpha=0.7)
axes[1].scatter(window_centers[iso_anomalies_window], iso_scores[iso_anomalies_window],
                color='red', s=100, label='Detected Anomalies', zorder=5)
axes[1].set_title('Isolation Forest Anomaly Scores', fontsize=14)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Anomaly Score (lower = more anomalous)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 評価
precision_iso = precision_score(true_labels, iso_anomalies_timeseries, zero_division=0)
recall_iso = recall_score(true_labels, iso_anomalies_timeseries, zero_division=0)
f1_iso = f1_score(true_labels, iso_anomalies_timeseries, zero_division=0)

print("=== ウィンドウベースIsolation Forestの評価 ===")
print(f"Precision: {precision_iso:.4f}")
print(f"Recall: {recall_iso:.4f}")
print(f"F1-Score: {f1_iso:.4f}")

## 6. 手法の比較

3つの手法を比較してみましょう。

In [ ]:
# 結果の比較
comparison_df = pd.DataFrame({
    'Method': ['Statistical (MA + 3σ)', 'Z-Score', 'Window-based Isolation Forest'],
    'Precision': [precision_stat, precision_z, precision_iso],
    'Recall': [recall_stat, recall_z, recall_iso],
    'F1-Score': [f1_stat, f1_z, f1_iso]
})

print("\n=== 手法の比較 ===")
print(comparison_df.round(4))

# 可視化
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(comparison_df))
width = 0.25

ax.bar(x - width, comparison_df['Precision'], width, label='Precision', alpha=0.8)
ax.bar(x, comparison_df['Recall'], width, label='Recall', alpha=0.8)
ax.bar(x + width, comparison_df['F1-Score'], width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Method', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Comparison of Time Series Anomaly Detection Methods', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Method'], rotation=15, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 3つの手法の検出結果を一つの図で比較
fig, axes = plt.subplots(4, 1, figsize=(14, 16))

# 真の異常
axes[0].plot(time, test_data, label='Data', color='blue', alpha=0.7)
axes[0].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5)
axes[0].set_title('Ground Truth (True Anomalies)', fontsize=14)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 統計的手法
axes[1].plot(time, test_data, label='Data', color='blue', alpha=0.3)
detected_stat = np.where(stat_anomalies_clean)[0]
axes[1].scatter(time[detected_stat], test_data[detected_stat], 
                color='orange', s=80, label='Detected', marker='x')
axes[1].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5, alpha=0.5)
axes[1].set_title(f'Statistical Method (MA + 3σ) - F1={f1_stat:.3f}', fontsize=14)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Z-Score
axes[2].plot(time, test_data, label='Data', color='blue', alpha=0.3)
axes[2].scatter(time[detected_z], test_data[detected_z], 
                color='purple', s=80, label='Detected', marker='x')
axes[2].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5, alpha=0.5)
axes[2].set_title(f'Z-Score Method - F1={f1_z:.3f}', fontsize=14)
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Value')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Isolation Forest
axes[3].plot(time, test_data, label='Data', color='blue', alpha=0.3)
for i, is_anomaly in enumerate(iso_anomalies_window):
    if is_anomaly:
        start = i
        end = i + WINDOW_SIZE
        axes[3].axvspan(time[start], time[min(end, len(time)-1)], 
                       alpha=0.3, color='green', label='Detected' if i == np.where(iso_anomalies_window)[0][0] else '')
axes[3].scatter(time[anomaly_indices], test_data[anomaly_indices], 
                color='red', s=150, label='True Anomalies', zorder=5, alpha=0.5)
axes[3].set_title(f'Window-based Isolation Forest - F1={f1_iso:.3f}', fontsize=14)
axes[3].set_xlabel('Time')
axes[3].set_ylabel('Value')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. まとめ

### 学んだこと

#### 1. 時系列異常検知の重要性
- 時系列データには**時間的な依存関係**がある
- 静的データ向けの手法だけでは不十分
- 過去のパターンを考慮する必要がある

#### 2. 3つの主要手法

**統計的手法（移動平均 + 標準偏差）**
- ✅ シンプルで理解しやすい
- ✅ 高速、リアルタイム処理に適している
- ❌ 複雑なパターンは検出困難
- **適用**: システム監視、簡単な異常検知

**Z-Score**
- ✅ 非常にシンプル
- ✅ 統計的に明確な基準
- ❌ 時間的依存関係を考慮しない
- **適用**: 簡易的な異常検知、前処理

**ウィンドウベース + Isolation Forest**
- ✅ 時間的な文脈を考慮
- ✅ 複雑なパターンも検出可能
- ✅ 特徴量エンジニアリングで柔軟性が高い
- ❌ やや複雑、計算コストがやや高い
- **適用**: DDoS攻撃検知、複雑な異常パターン

### 手法の選択基準

| 状況 | 推奨手法 |
|------|----------|
| リアルタイム処理が必須 | 統計的手法 |
| シンプルな異常（スパイクなど） | Z-Score、統計的手法 |
| 複雑なパターン | ウィンドウベース + Isolation Forest |
| 時間的依存関係が重要 | ウィンドウベース手法 |
| 非常に複雑なパターン | LSTM Autoencoder（深層学習） |

### DDoS攻撃検知への応用

#### 実務での活用例

```python
# トラフィックデータの時系列
時系列特徴量:
- リクエスト数/秒
- パケットサイズの平均
- 送信元IPの多様性
- レスポンス時間

# 多層検知システム
1. リアルタイム層（統計的手法）
   - 移動平均 + 3σで即座にアラート
   - 計算が高速、遅延なし

2. 詳細分析層（ウィンドウベース）
   - Isolation Forestで複雑なパターンを検出
   - 誤検知を減らす

3. 高度分析層（深層学習）
   - LSTM Autoencoderで新種の攻撃を検出
   - オフラインで詳細分析
```

### 次のステップ

#### 1. 実際のサイバー攻撃データセットで実践
- **KDD Cup 99**: 古典的なネットワーク侵入検知データ
- **NSL-KDD**: KDD Cup 99の改良版
- **CIC-IDS2017/2018**: 最新のサイバー攻撃データ
- **UNSW-NB15**: 現代的なネットワークトラフィックデータ

#### 2. 深層学習手法の学習
- **LSTM Autoencoder**: 時系列の正常パターンを学習
- **GRU Autoencoder**: LSTMの軽量版
- **Transformer**: 最新のアーキテクチャ
- **VAE (Variational Autoencoder)**: 確率的なAutoencoder

#### 3. 多変量時系列への拡張
- 複数の特徴量を同時に扱う
- 特徴量間の相関を考慮
- より複雑な異常パターンを検出

#### 4. リアルタイムシステムの構築
- ストリーミングデータへの対応
- オンライン学習
- アラートシステムとの統合

### 参考資料

- [scikit-learn: Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
- [Time Series Anomaly Detection](https://en.wikipedia.org/wiki/Anomaly_detection)
- [LSTM for Time Series](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/)
- [DDoS Detection using Machine Learning](https://arxiv.org/abs/1804.00932)